In [1]:
import torch 
from torch.utils.data import DataLoader
from src import full_pcd_dataset
import importlib
importlib.reload(full_pcd_dataset)
import json
# Create dataset instance
dataset = full_pcd_dataset.FullPCDDataset("data/full_pcd_150000_samples_6d_1.npz")
with open("data/label_dict.json", "r") as f:
    label_dict = json.load(f)
num_classes = len(label_dict.keys())
# Split into train and validation (example: 80-20 split)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Create data loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0,pin_memory=True)
num_classes

5

In [3]:
from src.model_updated import PointNet2Classification 
from torch.optim.lr_scheduler import StepLR
backbone_params = {"sa_mlps": [[2, 2, 2], [2, 2, 2]], "mlp":[64, 128, 256], "downsample_points": [256, 64], "radii":[0.1, 0.15], "ks":[16, 32], "add_xyz":True}

model = PointNet2Classification(num_classes, backbone_params=backbone_params)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5) 

In [4]:
from src.model_utils import count_parameters

print(f"Number of parameters in the model: {count_parameters(model)}")

Number of parameters in the model: 61001


In [5]:
from src.train_classificator import train_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

2025-05-26 15:22:55.254944: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748262175.275570   15827 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748262175.281326   15827 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748262175.297758   15827 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748262175.297781   15827 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748262175.297783   15827 computation_placer.cc:177] computation placer alr

Using device: cuda


In [6]:
train_model(model, train_loader, val_loader, optimizer, scheduler, device, epochs=100, directory="cls_updated_vsmall_new_data_2")

Training: 100%|██████████| 938/938 [02:52<00:00,  5.45it/s]


Epoch 1/100 - Total Loss: 1.1026 | Accuracy: 45.06%


Validating: 100%|██████████| 235/235 [00:38<00:00,  6.08it/s]


Validation - Loss: 0.8053 | Accuracy: 55.45%


Training: 100%|██████████| 938/938 [02:51<00:00,  5.46it/s]


...

Epoch 11/100 - Total Loss: 0.5859 | Accuracy: 66.62%


Validating: 100%|██████████| 235/235 [00:39<00:00,  6.02it/s]


Validation - Loss: 0.5428 | Accuracy: 69.10%


Training:   4%|▍         | 39/938 [00:07<02:51,  5.23it/s]


KeyboardInterrupt: 

In [7]:
checkpoint = torch.load('cls_updated_vsmall_new_data_1/checkpoint_epoch_10.pth', map_location='cpu')  # Load on CPU to avoid GPU memory issues
saved_state_dict = checkpoint['model_state']

# Print layer names and their shapes
for name, param in saved_state_dict.items():
    print(f"{name}: {param.shape}")

backbone.sa1.conv.0.0.weight: torch.Size([16, 3, 1, 1])
backbone.sa1.conv.0.1.weight: torch.Size([16])
backbone.sa1.conv.0.1.bias: torch.Size([16])
backbone.sa1.conv.0.1.running_mean: torch.Size([16])
backbone.sa1.conv.0.1.running_var: torch.Size([16])
backbone.sa1.conv.0.1.num_batches_tracked: torch.Size([])
backbone.sa1.conv.1.0.weight: torch.Size([16, 16, 1, 1])
backbone.sa1.conv.1.1.weight: torch.Size([16])
backbone.sa1.conv.1.1.bias: torch.Size([16])
backbone.sa1.conv.1.1.running_mean: torch.Size([16])
backbone.sa1.conv.1.1.running_var: torch.Size([16])
backbone.sa1.conv.1.1.num_batches_tracked: torch.Size([])
backbone.sa1.conv.2.weight: torch.Size([16, 16, 1, 1])
backbone.sa1.last_norm.weight: torch.Size([16])
backbone.sa1.last_norm.bias: torch.Size([16])
backbone.sa1.last_norm.running_mean: torch.Size([16])
backbone.sa1.last_norm.running_var: torch.Size([16])
backbone.sa1.last_norm.num_batches_tracked: torch.Size([])
backbone.sa2.conv.0.0.weight: torch.Size([16, 19, 1, 1])
backb